In [116]:
import cudaq
import numpy as np
from typing import Union, List
import math

In [117]:
def QR_decomposition(b):  

    b_norm = np.linalg.norm(b)
    N = len(b)

    B = np.column_stack((b / b_norm, np.random.randn(N, N - 1)))
        # Apply QR decomposition to B to get an orthonormal basis
        # The Q matrix from the QR decomposition will be unitary, and the first column will be b
    Q, _ = scipy.linalg.qr(B, mode='economic')
    return Q


@cudaq.kernel
def initialize_b( qb: cudaq.qvector , Q:list[float]):

    cudaq.compute_action("Q", Q)

    Q(qb)
    

In [118]:
@cudaq.kernel
def QFT(q):

    for i in range(len(q)):
        h(q[i])
        for j in range(i + 1, len(q)):
            angle = (2 * np.pi) / (2**(j - i + 1))
            cr1(angle, [q[j]], q[i])

    #cudaq.adjoint(quantum_fourier_transform, qubits)



@cudaq.kernel
def QPE(q_eigenvalue: cudaq.qvector, q_eigenvector: cudaq.qvector, U:list[float]):

    cudaq.register_operation("U", U)

    for i in range(len(q_eigenvalue)):
        h(q_eigenvalue[i])
   
    for i in range(len(q_eigenvalue)):
        # Apply U,  i times
        for j in range(0,2**i):
        # Apply the transformation U only if control qubit is in a |1> state to all the q_eigenvector qubits
            U.ctrl([q_eigenvalue[i]],q_eigenvector)

    QFT.adj(q_eigenvalue)

In [119]:

@cudaq.kernel
def controlled_rotations(q_eigenvalue: cudaq.qvector, q_flag: cudaq.qvector, thetas:list[float]):

    for i in range(q_eigenvalue):
        # Apply the transformation U only if control qubit is in a |1> state to all the q_eigenvector qubits
            rz(thetas[i]).ctrl(q_eigenvalue[i], q_flag)


In [120]:
@cudaq.kernel
def HHL(A:list[float],  b:list[float], thetas:list[float] ):
    """
    Construct the HHL circuit.

    Args:
        matrix: The matrix specifying the system, i.e., A in Ax=b.
        vector: The vector specifying the right-hand side of the equation in Ax=b.
        neg_vals: Indicates if the matrix has negative eigenvalues.

    Returns:
        A CUDA Quantum Kernel representing the HHL circuit.
    """

    nb = math.log(len(b),2)

    # Allocate quantum registers
    qb = kernel.qalloc(nb)   # Qubits for the vector
    ql = kernel.qalloc(nb)   # Qubits for eigenvalue evaluation
    qf = kernel.qalloc(1)    # Flag qubit

    #1. Load "b" (should I do it in binary form? No)
    # State preparation - normalize vector for state preparation
    # CUDAq does not provide direct isometry, apply approximate state prep.
    # Assuming `apply_vector_state()` handles this normalization
    # Initialize the quantum circuit to the initial state.

    Q = QR_decomposition(b)
    initialize_b(qb, Q)

    #2. QPE
    # Eigenvalue estimation placeholder
    # CUDAq does not directly support QPE; needs a manual phase estimation routine
    # Define QPE matrix-based gate equivalent for matrix with Trotterization
    # Apply Hadamard gates and controlled rotation gates.

    #create the unitary U=e^(iAt) 
    #Qubitization or Trotterization
    U = np.exp(j*A*2 * np.pi)
    QPE(ql, qb, U)


    #3. Ham sim to invert Eigenvalues
    # Hamiltonian simulation (e.g., Trotterization for matrix exponentiation)
    # Simplified to only consider a small power series for expm
    # Manually applying rotations, Z gates to simulate Hamiltonian evolution

    controlled_rotations(ql,qf, thetas)

    #4.QPE^
    # Inverse phase estimation
    # Apply the inverse rotation of QPE
    #cudaq.adjoint(QPE, qb, ql, U)
    QPE.adj(qb, ql, U)

In [121]:
n=10

A=np.ones((n,n))
b=np.ones(n)

lambdas,eigenvector = np.linalg.eig(A)
thetas = np.arcsin(np.min(lambdas) / lambdas)

hhl_kernel = HHL(A, b, thetas)

# Print the circuit
print(cudaq.draw(hhl_kernel))

/tmp/ipykernel_6508/1464425126.py:7: RuntimeWarning: invalid value encountered in divide
  thetas = np.arcsin(np.min(lambdas) / lambdas)


AttributeError: 'Call' object has no attribute 'id'